<a href="https://colab.research.google.com/github/Rishabh-Thapliyal/Google-colab/blob/main/SEO_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_hub
!pip install tensorflow_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 45.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard
    Fou

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
import tensorflow_text as text

from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.utils import pad_sequences, to_categorical
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('bbc-text.csv')

In [ ]:
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [ ]:
import re
from nltk import word_tokenize
import nltk
from nltk.corpus import stopwords

In [ ]:
space = re.compile('[/(){}\[\]\|@,;]')
symbols= re.compile('[^0-9a-z #+_]')
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = space.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = symbols.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing.
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df['clean_text'] = df['text'].apply(lambda x: clean_text(x))

In [ ]:
df.head()

,category,text,clean_text
0,tech,tv future in the hands of viewers with home th...,tv future hands viewers home theatre systems p...
1,business,worldcom boss left books alone former worldc...,worldcom boss left books alone former worldcom...
2,sport,tigers wary of farrell gamble leicester say ...,tigers wary farrell gamble leicester say rushe...
3,sport,yeading face newcastle in fa cup premiership s...,yeading face newcastle fa cup premiership side...
4,entertainment,ocean s twelve raids box office ocean s twelve...,ocean twelve raids bo office ocean twelve crim...


## **Defining X and Y**

In [ ]:
X = df['clean_text']

In [ ]:
encoder = LabelEncoder()
df['y_labels'] = encoder.fit_transform(df['category'])

In [ ]:
num_classes = np.max(df['y_labels']) + 1
Y = to_categorical(df['y_labels'], num_classes)

In [ ]:
Y

array([[0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.]], dtype=float32)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,)

## **BERT Model**

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
text = ['He is a good boy']
bert_preprocess(text)

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[ 101, 2002, 2003, 1037, 2204, 2879,  102,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,   

In [ ]:
text_2 = ['He is a good boy. '*129]
bert_preprocess(text_2)

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[ 101, 2002, 2003, 1037, 2204, 2879, 1012, 2002, 2003, 1037, 2204,
         2879, 1012, 2002, 2003, 1037, 2204, 2879, 1012, 2002, 2003, 1037,
         2204, 2879, 1012, 2002, 2003, 1037, 2204, 2879, 1012, 2002, 2003,
         1037, 2204, 2879, 1012, 2002, 2003, 1037, 2204, 2879, 1012, 2002,
         2003, 1037, 2204, 2879, 1012, 2002, 2003, 1037, 2204, 2879, 1012,
         2002, 2003, 1037, 22

In [ ]:
text_3 = ['He is a good boy. He likes to play cricket.']
bert_preprocess(text_3)

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[ 101, 2002, 2003, 1037, 2204, 2879, 1012, 2002, 7777, 2000, 2377,
         4533, 1012,  102,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,   

In [ ]:
text_4 = ['He is a good boy. He is a good boy.', 'He likes to play cricket.']
bert_preprocess(text_4)

{'input_type_ids': <tf.Tensor: shape=(2, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: s

In [ ]:
text_5 = ['He is a good boy. He likes nlp. Apple is good for health.', 'He likes to play cricket.']
bert_preprocess(text_5)

{'input_type_ids': <tf.Tensor: shape=(2, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: s

# *pooled_output represents each input sequence as a whole. The shape is [batch_size, H(768)].*

In [ ]:
text_3 = ['He is a good boy. He likes to play cricket.']
encoder_input = bert_preprocess(text_3)
encoder_output = bert_encoder(encoder_input)

In [ ]:
encoder_output['pooled_output']

<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[-0.8339926 , -0.62777406, -0.99314624,  0.79826087,  0.9216579 ,
        -0.26337138,  0.88795936,  0.46019325, -0.9527905 , -0.99999475,
        -0.8568339 ,  0.9805117 ,  0.98318386,  0.8200484 ,  0.9009741 ,
        -0.84533423, -0.64879733, -0.72058815,  0.26869774,  0.0134045 ,
         0.8372594 ,  1.        , -0.6138333 ,  0.43751884,  0.6042592 ,
         0.99846655, -0.88046455,  0.88441586,  0.9420868 ,  0.67003167,
        -0.5535924 ,  0.30296013, -0.9918719 , -0.31154117, -0.9950045 ,
        -0.99529606,  0.6760122 , -0.6199482 , -0.1243483 , -0.12329591,
        -0.8587389 ,  0.5916193 ,  0.99999803, -0.5761285 ,  0.74669224,
        -0.37812078, -1.        ,  0.4182452 , -0.8290352 ,  0.98487645,
         0.9851984 ,  0.9655208 ,  0.25691235,  0.5362659 ,  0.621403  ,
        -0.2857523 , -0.02681452,  0.11126145, -0.30835035, -0.67085284,
        -0.67023027,  0.58585894, -0.9541802 , -0.8869449 ,  0.9857477 ,
 

# **sequence_output** represents each input token in the context. The shape is [batch_size, seq_length, H]. Think of this as a contextual embedding for every token in the input.

In [ ]:
encoder_output['sequence_output'][0][0]

<tf.Tensor: shape=(768,), dtype=float32, numpy=
array([-2.37912178e-01,  5.29355891e-02, -2.25681797e-01, -6.42033890e-02,
       -2.39136234e-01, -4.16776955e-01,  4.14980084e-01,  6.12630665e-01,
       -1.43311352e-01, -3.16183656e-01, -1.03320107e-01, -1.83673322e-01,
        3.50163817e-01,  2.40038931e-01, -3.36565524e-02, -2.03271583e-01,
        9.33934748e-03,  6.75245166e-01,  2.78832465e-01, -2.10795149e-01,
       -9.11030993e-02, -2.35031590e-01, -6.56695515e-02, -1.20823272e-02,
       -3.97179537e-02, -1.10408008e-01,  5.21628447e-02, -8.16786364e-02,
        8.10483098e-02, -2.01672420e-01,  2.57016897e-01,  6.50955141e-01,
        6.06956519e-03, -1.94866415e-02, -1.86072528e-01, -1.10774457e-01,
        4.66272712e-01,  1.36448309e-01, -1.18121475e-01,  6.65305927e-02,
        1.07804358e-01,  2.61783987e-01, -9.00404006e-02, -1.00664943e-01,
        1.21517584e-01, -6.90401018e-01, -2.50761271e+00, -1.83599576e-01,
        3.11663985e-01, -6.46177232e-01,  2.82030970

## **encoder_outputs** are the intermediate activations of the L Transformer blocks. outputs["encoder_outputs"][i] is a Tensor of shape [batch_size, seq_length, H] with the outputs of the i-th Transformer block, for 0 <= i < L. The last value of the list is equal to sequence_output.

In [ ]:
encoder_output['encoder_outputs'][1]

<tf.Tensor: shape=(1, 128, 768), dtype=float32, numpy=
array([[[-0.11928062, -0.17994747, -0.43276381, ...,  0.38110328,
          0.08531247, -0.0486607 ],
        [ 0.13117729,  0.14938203, -0.01590831, ..., -0.01273311,
          0.04214494, -0.2355211 ],
        [-0.5583645 , -0.6351034 ,  0.19882527, ...,  0.27039865,
          0.20288013,  0.330382  ],
        ...,
        [-0.07337263, -0.41901815,  0.8468604 , ...,  1.2136645 ,
         -0.4649717 , -0.0825286 ],
        [-0.08730207, -0.3855874 ,  0.81720287, ...,  1.3039207 ,
         -0.52225673, -0.22670738],
        [-0.00192555, -0.47019008,  0.6596734 , ...,  1.6548862 ,
         -0.71750766, -0.23823959]]], dtype=float32)>

## Encoder output for last layer equals sequence output

In [ ]:
encoder_output['encoder_outputs'][11]  # this is same as sequence_output

<tf.Tensor: shape=(1, 128, 768), dtype=float32, numpy=
array([[[-3.3577326e-01, -1.0569543e-01, -1.6116691e-01, ...,
         -2.7213454e-01,  9.8831022e-01,  9.0782416e-01],
        [ 1.0829307e-03, -2.9758978e-01, -4.5995674e-01, ...,
         -8.6016156e-02,  1.3794137e+00, -5.5146016e-02],
        [ 2.6117366e-02, -3.3422056e-01,  2.2623929e-01, ...,
         -1.8393686e-01,  5.4771048e-01,  6.4975578e-01],
        ...,
        [-1.7558172e-01, -8.5478801e-01,  4.5054931e-02, ...,
          5.8065701e-01,  3.8624433e-01, -2.3509446e-01],
        [ 4.1395292e-02, -4.9673015e-01,  3.4354800e-01, ...,
          5.9165454e-01,  2.0361429e-01, -2.0535322e-01],
        [-2.0106214e-01, -9.0469825e-01,  1.8796402e-01, ...,
          5.8163577e-01,  2.7740127e-01, -2.7108860e-01]]], dtype=float32)>

In [ ]:
# BERT layers
# functional models

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])  # as our task is fine-tuning
l = tf.keras.layers.Dense(num_classes, activation='softmax', name="output")(l)

model = tf.keras.Model(inputs=[text_input], outputs = [l])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_word_ids': (None,                                            
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             }                                                                

In [ ]:
model.compile(optimizer='adam',
               loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, Y_train, epochs = 7)

Epoch 1/7
53/53 [==============================] - 29s 338ms/step - loss: 1.5015 - accuracy: 0.3633
Epoch 2/7
53/53 [==============================] - 18s 341ms/step - loss: 1.2191 - accuracy: 0.5570
Epoch 3/7
53/53 [==============================] - 19s 355ms/step - loss: 1.0337 - accuracy: 0.7038
Epoch 4/7
53/53 [==============================] - 19s 364ms/step - loss: 0.9129 - accuracy: 0.7560
Epoch 5/7
53/53 [==============================] - 19s 361ms/step - loss: 0.8109 - accuracy: 0.7998
Epoch 6/7
53/53 [==============================] - 19s 365ms/step - loss: 0.7231 - accuracy: 0.8291
Epoch 7/7
53/53 [==============================] - 19s 359ms/step - loss: 0.6569 - accuracy: 0.8555


In [ ]:
model.evaluate(X_test, Y_test)

18/18 [==============================] - 7s 367ms/step - loss: 0.6618 - accuracy: 0.8564


[0.6617609262466431, 0.8563734292984009]

In [ ]:
model.predict(X_test)

18/18 [==============================] - 7s 332ms/step


array([[0.01556203, 0.12366281, 0.02264808, 0.8257449 , 0.01238224],
       [0.36798853, 0.04152415, 0.33346498, 0.02518967, 0.2318327 ],
       [0.15122233, 0.10506303, 0.08066668, 0.13786286, 0.5251851 ],
       ...,
       [0.04294531, 0.31921303, 0.07855722, 0.52625364, 0.03303074],
       [0.02106411, 0.63245046, 0.0616404 , 0.0532679 , 0.23157707],
       [0.1317269 , 0.03183733, 0.13036029, 0.00613348, 0.69994205]],
      dtype=float32)

In [ ]:
Y_test

array([[0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.]], dtype=float32)